# OSM Shapefile Decoding

In [1]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-21 11:45:03.285980


In [2]:
# Global Params
basedir="/Users/tgadf/Downloads/osm"
prec=7
mainkey="OSM"

# Parse OSM Data

In [3]:
from ioUtils import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

fastfood=["fastfood", "foodcourt","convenience", "cafe"]
restaurant=["restaurant"]
homefood=["greengrocer", "pharmacy", "supermarket", "chemist"]
lodging=["hotel", "motel", "hostel","chalet","bedandbreakfast","guesthouse", "caravansite"]
commercial=["bookshop","butcher","hairdresser", "furnitureshop", "doityourself", "gardencentre", "newsagent", "toyshop","mall","laundry","mobilephoneshop","bank","atm"]
commercial+=["bicyclerental","bicycleshop","bakery","beautyshop","videoshop","outdoorshop", 'jeweller','kiosk', 'shoeshop','sportsshop', "travelagent"]
commercial+=['florist','giftshop', "departmentstore", "computershop"]
vending=['vendingany','vendingcigarette','vendingmachine','vendingparking']
commercial+=vending
auto=['cardealership','carrental','carrepair','carsharing','carwash']
municipal=["firestation", "prison", "library", "postoffice", "postbox", "police", "townhall","communitycentre","artscentre", "publicbuilding", "courthouse"]
industrial=["wastewaterplant", "windmill","watermill","waterworks", "recycling", "recyclingclothes", "recyclingglass", "recyclingmetal", "recyclingpaper", "commstower", 'watertower','waterwell', "tower"]
building=["embassy"]
medical=["hospital", "dentist", "doctors", "optician", "veterinary","nursinghome"]
attraction=["attraction","fountain", "museum", "touristinfo", "viewpoint", "zoo","themepark","monument","picnicsite","fort","castle","battlefield","archaeological","memorial", "lighthouse", "observationtower", "artwork"]
religious=["graveyard","waysidecross","waysideshrine","ruins"]
sport=["golfcourse", "huntingstand", "icerink", "sportscentre", "track", "swimmingpool"]
entertainment=["bar", "pub", "nightclub", "theatre", "cinema"]
recreation=["stadium", "campsite","biergarten","dogpark","park", "pitch", "playground"]
school=["kindergarten","school"]
college=["college","university"]
general=['alpinehut','bench','beverages','camerasurveillance','clothes','drinkingwater','shelter','stationery','telephone','toilet','wastebasket']                

increase={}

def parseOSM(dirval, prec, key=""):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*{0}*.shp".format(key)))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    
    
    shapeData   = {}
    geoShapeMap = {}
    ngeos     = 0
    totalgeos = 0
    show      = False
    cntr      = Counter()
        
    print("Found the following {0} shape files: {1}".format(len(osms), ", ".join(osms)))
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, prec)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        Nshapes   = len(sf.shapes())

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 2500 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                if fclass.find("christian_") != -1:
                    fclass = "christian"
                elif fclass.find("muslim_") != -1:
                    fclass = "muslim"
                else:
                    fclass = fclass
                geoid = fclass
            elif osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("bicycle") != -1:
                    fclass = "bicycle"
                elif fclass.find("parking_") != -1:
                    fclass = "parking"
                elif fclass.find("mini_roundabout") != -1 or fclass.find("turning_circle") != -1:
                    fclass = "roundabout"
                elif fclass.find("speed_camera") != -1:
                    fclass = "speedcamera"
                elif fclass.find("motorway_junction") != -1:
                    fclass = "ramp"
                elif fclass.find("lock_gate") != -1:
                    fclass = "gate"
                elif fclass.find("traffic_signals") != -1:
                    fclass = "signal"
                elif fclass.find("street_lamp") != -1:
                    fclass = "lamp"
                else:
                    fclass = fclass
                    
                if fclass not in ["parking","fuel"]:
                    continue
                geoid = fclass
            elif osm in ["gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("railway_") != -1:
                    fclass = fclass.replace("railway", "rail")
                fclass = fclass.replace("_", "")
                
                if fclass.startswith("bus"):
                    fclass = "bus"
                elif fclass.startswith("rail"):
                    fclass = "rail"
                elif fclass.startswith("ferry"):
                    fclass = "ferry"
                elif fclass.startswith("tram"):
                    fclass = "tram"
                elif fclass.startswith("taxi"):
                    fclass = "taxi"
                else:
                    raise ValueError("fClass {0} was not recognized!".format(fclass))
                
                geoid = fclass
            elif osm in ["gis_osm_pois_a_free_1", "gis_osm_pois_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                fclass = fclass.replace("_", "")
                
                if fclass in fastfood:
                    fclass = "fastfood"
                elif fclass in restaurant:
                    fclass = "restaurant"
                elif fclass in homefood:
                    fclass = "grocery"
                elif fclass in lodging:
                    fclass = "lodging"
                elif fclass in commercial:
                    fclass = "commercial"
                elif fclass in auto:
                    fclass = "auto"
                elif fclass in municipal:
                    fclass = "municipal"
                elif fclass in industrial:
                    fclass = "industrial"
                elif fclass in building:
                    fclass = "building"
                elif fclass in medical:
                    fclass = "medical"
                elif fclass in attraction:
                    fclass = "attraction"
                elif fclass in religious:
                    fclass = "religious"
                elif fclass in sport:
                    fclass = "sport"
                elif fclass in entertainment:
                    fclass = "entertainment"
                elif fclass in recreation:
                    fclass = "recreation"
                elif fclass in school:
                    fclass = "school"
                elif fclass in college:
                    fclass = "college"
                elif fclass in general:
                    fclass = "public"
                else:
                    raise ValueError("fClass {0} was not found!".format(fclass))
                geoid = fclass
            elif osm in ["gis_osm_water_a_free_1", "gis_osm_waterways_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_natural_a_free_1", "gis_osm_natural_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_landuse_a_free_1", "gis_osm_landuse_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #print(irec,fclass,geoid)
            #continue
            #if irec > 1000: 1/0

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            
            geos = getGeos(shape, prec=prec, linear=False, returnKeys=True, debug=False)

            if False:
                useLinear = False
                if len(points) == 1 or useLinear is True:
                    geos = getGeos(shape, prec=prec, linear=True, returnKeys=True)
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
                else:
                    geos = getInitGeo(shape)
                    geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                    if geos is None:
                        useLinear = True
                        geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = 1
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        print("Finished: {0}".format(cntr.most_common(100)))
        print("\n")
            
    found = False
    for k,v in cntr.most_common():
        if k.find(" ") != -1 or k.find(";") != -1 or k[0].isupper():
            print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
            found = True
    print("\n")
    for k,v in nameCntr.most_common(200):
        print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
    if len(shapeData) > 0:
        for geoid in geoShapeMap.keys():
            geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
        print("Found {0} geos from {1}".format(ngeos, shapeval))
        saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, key, prec)))

In [4]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    if exists(dirval):
        print("Directory {0} already exists".format(dirval))
        rmZipDir(dirval)
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    print("Extract {0}".format(zipname))
    zip_ref.extractall(dirval)
    zip_ref.close()
    
    #try:
    if True:
        parseOSM(dirval, 7, 'landuse')
    #except:
        rmZipDir(dirval)
    
    rmZipDir(dirval)

Extract /Users/tgadf/Downloads/osm/alabama-latest-free.shp.zip
Found the following 1 shape files: gis_osm_landuse_a_free_1
gis_osm_landuse_a_free_1
Current Time is Wed Nov 21, 2018 11:45:11 for 

Analyzing gis_osm_landuse_a_free_1	10484
Processed 2500/10484 records. Found 49499 geos so far...
Processed 5000/10484 records. Found 75362 geos so far...
Processed 7500/10484 records. Found 96216 geos so far...
Processed 10000/10484 records. Found 142509 geos so far...
Finished: [('forest', 2787), ('residential', 1741), ('grass', 1413), ('park', 1027), ('farm', 688), ('meadow', 610), ('cemetery', 577), ('industrial', 437), ('retail', 412), ('commercial', 408), ('scrub', 156), ('recreation_ground', 88), ('quarry', 54), ('nature_reserve', 43), ('military', 31), ('orchard', 5), ('heath', 4), ('allotments', 3)]




Found 145851 geos from gis_osm_landuse_a_free_1


There are 18 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/alabama-landuse-7-data.p


There are 145851 ent

Processed 60000/357054 records. Found 464987 geos so far...
Processed 62500/357054 records. Found 474533 geos so far...
Processed 65000/357054 records. Found 484172 geos so far...
Processed 67500/357054 records. Found 494468 geos so far...
Processed 70000/357054 records. Found 504944 geos so far...
Processed 72500/357054 records. Found 515891 geos so far...
Processed 75000/357054 records. Found 527681 geos so far...
Processed 77500/357054 records. Found 538531 geos so far...
Processed 80000/357054 records. Found 550256 geos so far...
Processed 82500/357054 records. Found 562112 geos so far...
Processed 85000/357054 records. Found 571891 geos so far...
Processed 87500/357054 records. Found 579800 geos so far...
Processed 90000/357054 records. Found 607943 geos so far...
Processed 92500/357054 records. Found 621709 geos so far...
Processed 95000/357054 records. Found 626406 geos so far...
Processed 97500/357054 records. Found 632847 geos so far...
Processed 100000/357054 records. Found 6

Current Time is Wed Nov 21, 2018 11:51:22 for 

Analyzing gis_osm_landuse_a_free_1	94727
Processed 2500/94727 records. Found 17163 geos so far...
Processed 5000/94727 records. Found 46101 geos so far...
Processed 7500/94727 records. Found 68768 geos so far...
Processed 10000/94727 records. Found 105500 geos so far...
Processed 12500/94727 records. Found 133854 geos so far...
Processed 15000/94727 records. Found 171137 geos so far...
Processed 17500/94727 records. Found 198185 geos so far...
Processed 20000/94727 records. Found 228123 geos so far...
Processed 22500/94727 records. Found 246982 geos so far...
Processed 25000/94727 records. Found 251965 geos so far...
Processed 27500/94727 records. Found 257800 geos so far...
Processed 30000/94727 records. Found 261723 geos so far...
Processed 32500/94727 records. Found 266191 geos so far...
Processed 35000/94727 records. Found 275424 geos so far...
Processed 37500/94727 records. Found 282710 geos so far...
Processed 40000/94727 records. F

Found the following 1 shape files: gis_osm_landuse_a_free_1
gis_osm_landuse_a_free_1
Current Time is Wed Nov 21, 2018 11:53:28 for 

Analyzing gis_osm_landuse_a_free_1	39300
Processed 2500/39300 records. Found 82402 geos so far...
Processed 5000/39300 records. Found 193933 geos so far...
Processed 7500/39300 records. Found 282351 geos so far...
Processed 10000/39300 records. Found 352238 geos so far...
Processed 12500/39300 records. Found 386980 geos so far...
Processed 15000/39300 records. Found 451296 geos so far...
Processed 17500/39300 records. Found 513446 geos so far...
Processed 20000/39300 records. Found 565768 geos so far...
Processed 22500/39300 records. Found 627519 geos so far...
Processed 25000/39300 records. Found 700633 geos so far...
Processed 27500/39300 records. Found 742006 geos so far...
Processed 30000/39300 records. Found 826090 geos so far...
Processed 32500/39300 records. Found 893808 geos so far...
Processed 35000/39300 records. Found 937645 geos so far...
Proc

Processed 17500/23255 records. Found 236803 geos so far...
Processed 20000/23255 records. Found 259950 geos so far...
Processed 22500/23255 records. Found 297013 geos so far...
Finished: [('farm', 4922), ('forest', 3883), ('residential', 3158), ('park', 2640), ('grass', 2632), ('cemetery', 1611), ('scrub', 1490), ('commercial', 910), ('nature_reserve', 769), ('retail', 417), ('industrial', 357), ('meadow', 205), ('quarry', 138), ('recreation_ground', 80), ('orchard', 20), ('military', 13), ('vineyard', 6), ('allotments', 3), ('heath', 1)]




Found 305229 geos from gis_osm_landuse_a_free_1


There are 19 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/iowa-landuse-7-data.p


There are 305229 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/iowa-landuse-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/iowa-latest-free.shp
Extract /Users/tgadf/Downloads/osm/kansas-latest-free.shp.zip
Found the following 1 shape files: gis_osm_land

...

Found the following 1 shape files: gis_osm_landuse_a_free_1
gis_osm_landuse_a_free_1
Current Time is Wed Nov 21, 2018 12:00:42 for 

Analyzing gis_osm_landuse_a_free_1	10168
Processed 2500/10168 records. Found 22397 geos so far...
Processed 5000/10168 records. Found 42569 geos so far...
Processed 7500/10168 records. Found 78158 geos so far...
Processed 10000/10168 records. Found 107804 geos so far...
Finished: [('farm', 4147), ('forest', 2093), ('park', 1212), ('grass', 968), ('residential', 452), ('cemetery', 393), ('recreation_ground', 261), ('industrial', 170), ('retail', 130), ('military', 107), ('meadow', 85), ('commercial', 61), ('orchard', 35), ('scrub', 20), ('quarry', 14), ('nature_reserve', 13), ('allotments', 4), ('vineyard', 3)]




Found 109593 geos from gis_osm_landuse_a_free_1


There are 18 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/nebraska-landuse-7-data.p


There are 109593 entries in the saved file.
Saved shape data to /Users/tgadf/Dow

Saved shape data to /Users/tgadf/Downloads/osm/new-york-landuse-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/new-york-latest-free.shp
Extract /Users/tgadf/Downloads/osm/north-carolina-latest-free.shp.zip
Found the following 1 shape files: gis_osm_landuse_a_free_1
gis_osm_landuse_a_free_1
Current Time is Wed Nov 21, 2018 12:03:46 for 

Analyzing gis_osm_landuse_a_free_1	24444
Processed 2500/24444 records. Found 81570 geos so far...
Processed 5000/24444 records. Found 109250 geos so far...
Processed 7500/24444 records. Found 133636 geos so far...
Processed 10000/24444 records. Found 155830 geos so far...
Processed 12500/24444 records. Found 188574 geos so far...
Processed 15000/24444 records. Found 215663 geos so far...
Processed 17500/24444 records. Found 268731 geos so far...
Processed 20000/24444 records. Found 283413 geos so far...
Processed 22500/24444 records. Found 353476 geos so far...
Finished: [('forest', 6660), ('residential', 4710), ('grass', 3608), ('park', 2624),

Processed 25000/46968 records. Found 646871 geos so far...
Processed 27500/46968 records. Found 685355 geos so far...
Processed 30000/46968 records. Found 715701 geos so far...
Processed 32500/46968 records. Found 738472 geos so far...
Processed 35000/46968 records. Found 796664 geos so far...
Processed 37500/46968 records. Found 834552 geos so far...
Processed 40000/46968 records. Found 873530 geos so far...
Processed 42500/46968 records. Found 921940 geos so far...
Processed 45000/46968 records. Found 940364 geos so far...
Finished: [('forest', 12543), ('grass', 11070), ('residential', 4696), ('farm', 4553), ('park', 4196), ('cemetery', 2076), ('scrub', 1475), ('meadow', 1444), ('industrial', 1306), ('retail', 1139), ('commercial', 1125), ('nature_reserve', 521), ('recreation_ground', 448), ('quarry', 199), ('orchard', 70), ('military', 53), ('allotments', 27), ('heath', 19), ('vineyard', 8)]




Found 964236 geos from gis_osm_landuse_a_free_1


There are 19 entries in the saved file

Processed 50000/54938 records. Found 458367 geos so far...
Processed 52500/54938 records. Found 478974 geos so far...
Finished: [('residential', 11896), ('grass', 10819), ('cemetery', 8036), ('park', 7152), ('forest', 4195), ('retail', 3230), ('commercial', 3133), ('industrial', 2422), ('farm', 1396), ('scrub', 907), ('meadow', 718), ('recreation_ground', 485), ('quarry', 184), ('nature_reserve', 151), ('orchard', 87), ('military', 54), ('vineyard', 30), ('heath', 28), ('allotments', 15)]




Found 502995 geos from gis_osm_landuse_a_free_1


There are 19 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/texas-landuse-7-data.p


There are 502995 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/texas-landuse-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/texas-latest-free.shp
Extract /Users/tgadf/Downloads/osm/utah-latest-free.shp.zip
Found the following 1 shape files: gis_osm_landuse_a_free_1
gis_osm_landuse_a_free_1
Current Time

Processed 10000/63768 records. Found 125288 geos so far...
Processed 12500/63768 records. Found 156881 geos so far...
Processed 15000/63768 records. Found 186755 geos so far...
Processed 17500/63768 records. Found 209015 geos so far...
Processed 20000/63768 records. Found 236534 geos so far...
Processed 22500/63768 records. Found 263797 geos so far...
Processed 25000/63768 records. Found 286254 geos so far...
Processed 27500/63768 records. Found 309103 geos so far...
Processed 30000/63768 records. Found 328306 geos so far...
Processed 32500/63768 records. Found 339132 geos so far...
Processed 35000/63768 records. Found 352054 geos so far...
Processed 37500/63768 records. Found 373013 geos so far...
Processed 40000/63768 records. Found 414342 geos so far...
Processed 42500/63768 records. Found 459191 geos so far...
Processed 45000/63768 records. Found 489583 geos so far...
Processed 47500/63768 records. Found 518505 geos so far...
Processed 50000/63768 records. Found 549265 geos so far.

In [24]:
# gis_osm_buildings_a_free_1.shp	gis_osm_pofw_a_free_1.shp	gis_osm_traffic_a_free_1.shp
# gis_osm_landuse_a_free_1.shp	gis_osm_pofw_free_1.shp		gis_osm_traffic_free_1.shp
# gis_osm_natural_a_free_1.shp	gis_osm_pois_a_free_1.shp	gis_osm_transport_a_free_1.shp
# gis_osm_natural_free_1.shp	gis_osm_pois_free_1.shp		gis_osm_transport_free_1.shp
# gis_osm_places_a_free_1.shp	gis_osm_railways_free_1.shp	gis_osm_water_a_free_1.shp
# gis_osm_places_free_1.shp	gis_osm_roads_free_1.shp	gis_osm_waterways_free_1.shp